In [41]:
#協定(protocal)，因為實作了__len__以及
#__getitem__符合序列(squence)的協定
#所以此Vector是序列

from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components) 

    def __repr__(self):
        components = reprlib.repr(self._components) #超過六個省略
        components = components[components.find('['):-1]  #切除不必要字元
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        hashes = (hash(x) for x in self) #產生器表達式
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self)) 

    def __bool__(self):
        return bool(abs(self))

    #One of Sequence Protocol
    def __len__(self):
        return len(self._components)
    #One of Sequence Protocol
    def __getitem__(self, index):
        cls = type(self) 
        if isinstance(index, slice): #如果index是個slice ex.v[1:3]
            return cls(self._components[index]) #回傳新的Vector物件
        elif isinstance(index, numbers.Integral): #如果index是個int ex.v[3]
            return self._components[index]  #回傳特定項目
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls)) 

    #設計__getattr__
    shortcut_names = 'xyzt'
    def __getattr__(self, name):
        cls = type(self)  
        if len(name) == 1:  
            pos = cls.shortcut_names.find(name) 
            if 0 <= pos < len(self._components):  
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'  
        raise AttributeError(msg.format(cls, name))

    #設計__setattr__，避免__getattr__的不一致性
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1: 
            if name in cls.shortcut_names:  
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  
            if error: 
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value) 

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented

    def __rmul__(self, scalar):
        return self * scalar
    
    #內積
    def __matmul__(self, other):
        try:
            return sum(a * b for a, b in zip(self, other))
        except TypeError:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other  # this only works in Python 3.5

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) 

In [4]:
#__repr__
Vector([3.1, 4.2]), Vector((3, 4, 5)), Vector(range(10))
#超過6個用...表示，因為reprlib

(Vector([3.1, 4.2]),
 Vector([3.0, 4.0, 5.0]),
 Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...]))

In [5]:
#__iter__
v1 = Vector([3, 4, 5])
x, y, z = v1
x, y, z

(3.0, 4.0, 5.0)

In [6]:
#__str__
v2 = Vector([3, 4, 5])
print(v2)

(3.0, 4.0, 5.0)


In [7]:
#__bool__
bool(Vector([3, 0, 0])), bool(Vector([0, 0, 0]))

(True, False)

In [8]:
# __getititem__
v3 = Vector(range(7))
v3[2:7:2], v3[4]

(Vector([2.0, 4.0, 6.0]), 4.0)

In [27]:
#__getattr__
v4 = Vector(range(1,11,2))
v4.x, v4.y, v4.z, v4.t, v4[4]

(1.0, 3.0, 5.0, 7.0, 9.0)

In [31]:
#__hash__
hash(Vector([1,2,3,4])),hash(Vector([1.1,2,3,4]))

(4, 230584300921369604)

In [34]:
#__format__
v4 = Vector([3, 4, 5])
format(v4)

'(3.0, 4.0, 5.0)'

In [37]:
#__format__
v5 = Vector([3, 4, 5])
format(v5, 'h')

'<7.0710678118654755, 1.1326472962107264, 0.8960553845713439>'

In [38]:
#__format__
v6 = Vector([3, 4, 5])
format(v6, '0.5fh')

'<7.07107, 1.13265, 0.89606>'

In [47]:
#算數overload
a = Vector([3, 4, 5])
b = Vector([6, 7, 8])
c = Vector([1, 2])

a + b, a*2 , a + c , a@b

(Vector([9.0, 11.0, 13.0]),
 Vector([6.0, 8.0, 10.0]),
 Vector([4.0, 6.0, 5.0]),
 86.0)